# Plotting odometry histograms

This notebook builds on the "Plotting odometry data" notebook that you were working with earlier. In addition to plotting timeseries we will explore the data by plotting histograms, which accumulates data over time to visualize how samples are distributed. We will work with the same data set as before.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import pupil_recording_interface as pri

## Load data

In [ ]:
folder = Path("/home/data/odometry/")
subject = "example_session"

Below, we use pupil_recording_interface to load the data

Note: this is a large file, and this may take a while! (up to 10 mins).

In [ ]:
accel = pri.load_dataset(folder / subject, accel="recording", cache=False)
gyro = pri.load_dataset(folder / subject, gyro="recording", cache=False)
odometry = pri.load_dataset(folder / subject, odometry="recording", cache=False)

As before, we will first look at the raw data from the Realsense gyroscope. First we will plot the full time series.


In [ ]:
gyro

In [ ]:
plt.plot(gyro.time,gyro.angular_velocity)
plt.legend(['x','y','z'])

Now we accumulate the data over time to see how it is distributed for each axis/dimension (x,y,z). We use the matplotlib hist function and specify 100 bins within the range of -2.5 to +2.5 rad/s.

In [ ]:
plt.subplot(131)
plt.hist(gyro.angular_velocity.sel(cartesian_axis='x'),100,[-2.5,2.5])
plt.subplot(132)
plt.hist(odometry.angular_velocity.sel(cartesian_axis='y'),100,[-2.5,2.5])
plt.subplot(133)
plt.hist(odometry.angular_velocity.sel(cartesian_axis='z'),100,[-2.5,2.5]);

What we learn from this is that angular velocity along a given axis is very often close to zero and is rarely larger than 1 rad/sec.

OK, let's try the same thing with the accelerometer data.

In [ ]:
accel

In [ ]:
plt.plot(accel.time,accel.linear_acceleration)
plt.legend(['x','y','z']);

We make histograms with 100 bins again, but we select a different range for each of the axes.

In [ ]:
plt.subplot(131)
plt.hist(accel.linear_acceleration.sel(cartesian_axis='x'),100,[-10,10])
plt.subplot(132)
plt.hist(accel.linear_acceleration.sel(cartesian_axis='y'),100,[0,20])
plt.subplot(133)
plt.hist(accel.linear_acceleration.sel(cartesian_axis='z'),100,[-10,10]);

We learn that gravity is acting most often along y-axis, which shows a peak in the histogram near 9.8 m/s^2. We can also observe that these histograms are asymmetrical. Given that the total acceleration is often dominated by gravitational acceleration, what might be causing this assymetry?

OK, now lets work with some odometry data.

In [ ]:
odometry

We have a separate estimate of angular velocity available here. Let's plot it and see whether it differs much from the gyro data.

In [ ]:
plt.plot(odometry.time,odometry.angular_velocity)
plt.legend(['x','y','z']);

In [ ]:
plt.subplot(131)
plt.hist(gyro.angular_velocity.sel(cartesian_axis='x'),100,[-2.5,2.5])
plt.subplot(132)
plt.hist(odometry.angular_velocity.sel(cartesian_axis='y'),100,[-2.5,2.5])
plt.subplot(133)
plt.hist(odometry.angular_velocity.sel(cartesian_axis='z'),100,[-2.5,2.5]);

There is a gap toward the end of the timeseries plot. This suggests that the odometry suffered some kind of failure during this time segment, but gyro data was still collected. The histograms looks very similar.

OK, now lets check out the linear velocity estimate, which we only get from the odometry.

In [ ]:
plt.plot(odometry.time,odometry.linear_velocity)
plt.legend(['x','y','z'])

In [ ]:
plt.subplot(131)
plt.hist(odometry.linear_velocity.sel(cartesian_axis='x'),100,[-2,2])
plt.subplot(132)
plt.hist(odometry.linear_velocity.sel(cartesian_axis='y'),100,[-2,2])
plt.subplot(133)
plt.hist(odometry.linear_velocity.sel(cartesian_axis='z'),100,[-2,2])

The timeseries plot is hard to inspect due to the large transients that occur near the end of the recording. Can you figure out how to recale the y-axis to better inspect this data?

The histograms show that data for x and z axes are similarly distributed, whereas y-axis data (velocity in the vertical direction) shows much less variance. But recall that this raw odometry data is different from the processed data. Let's look at that now.

In [ ]:
accel_calib = xr.open_dataset(folder / subject / "acceleration.nc")

In [ ]:
accel_calib

In [ ]:
plt.plot(accel_calib.time,accel_calib.linear_vel)
plt.legend(['x','y','z'])

In [ ]:
plt.subplot(131)
plt.hist(accel_calib.linear_vel.sel(cartesian_axis='x'),100,[-2,2])
plt.subplot(132)
plt.hist(accel_calib.linear_vel.sel(cartesian_axis='y'),100,[-2,2])
plt.subplot(133)
plt.hist(accel_calib.linear_vel.sel(cartesian_axis='z'),100,[-2,2]);

Notice that these histograms have a very different shape. The histogram for the x-axis linear velocity is bimodal, with a second peak near 1 m/s. And  the y and z-axis histograms show a similar distribution. What can account for these changes relative to the unprocessed linear velocity data?

Now, we will generate a 2D histogram of the y and z axis linear velocity data. 

In [ ]:
plt.hist2d(accel_calib.linear_vel.sel(cartesian_axis='z'),accel_calib.linear_vel.sel(cartesian_axis='y'),100,[[-.1,.1],[-.1,.1]]);

This histogram shows the components of linear velocity acting along the head-horiztonal (inter-aural) and head-vertical axes. This is a proxy for the direction of linear velocity in the frontal plane, also known as the heading direction. It is centered around straight ahead, because humans face forward in the direction they are walking, but with some variability due to head bob and shake during walking. 

We can learn something similar by examining the gravitational acceleration field of the processed accelerometer data. This data indicates how the components of the gravity vector (9.8 m/s^2) are distributed along the head-centered x, y, and z axes. From this data, we can learn how the head is tilted relative to gravity.

In [ ]:
plt.subplot(131)
plt.hist(accel_calib.gravitational_acc.sel(cartesian_axis='x'),100,[-10,10])
plt.subplot(132)
plt.hist(accel_calib.gravitational_acc.sel(cartesian_axis='y'),100,[-10,10])
plt.subplot(133)
plt.hist(accel_calib.gravitational_acc.sel(cartesian_axis='z'),100,[-10,10]);

In [ ]:
plt.hist2d(accel_calib.gravitational_acc.sel(cartesian_axis='y'),accel_calib.gravitational_acc.sel(cartesian_axis='x'),100)

The vertical and horiztonal axes show how the head is pitched and rolled realtive to gravity. The vertical distrbituions is skewed toward the downward direction, that is downward head tilt.